In [1]:
import numpy as np
import scipy.io as scio
import os
import librosa
import generateSegmentationData as gsd
import pathlib
import scipy.io as scio
import pickle
import pandas 
from trainSpringerSegmentationAlgorithm import *
from runSpringerSegmentationAlgorithm import *
from plot_figs import save_figs
from undoSegmentationData import *

## Load the default options:
# These options control options such as the original sampling frequency of
# the data, the sampling frequency for the derived features and whether the
# mex code should be used for the Viterbi decoding:

springer_options = default_Springer_HSMM_options(SpringerOptions)

## Carrega o modelo
filepath = '/home/linse/stevan/Qualirede/segmentador/springerList.pkl'
pkl_file = open(filepath, 'rb')
model = pickle.load(pkl_file)
pkl_file.close()

## Define os paths
Root = '/media/linse/DADOS/stevan/datasets/heart-sound'
Root = pathlib.Path(Root)

inputPath = Root/'training'
outputPath = Root/'python_segmentation/'
outputPath.mkdir(parents=True, exist_ok=True)

segPath1 = Root/'extra_data/hand_corrected_annotations_training/'
segPath2 = Root/'extra_data/automated_Springer_annotations_training/'

figsPath = '/home/linse/stevan/Qualirede/segmentador/figures'
figsPath = pathlib.Path(figsPath)

metadataPath = '/media/linse/DADOS/stevan/datasets/heart-sound/training/metadata.csv'
df = pandas.read_csv(metadataPath)
quality = df['quality']

## Coloca as listas em ordem alfabética
folderList = sorted(list(inputPath.glob('training*')))
segFolderList1 = sorted(list(segPath1.glob('training*')))
segFolderList2 = sorted(list(segPath2.glob('training*')))

# 'training-a/', 'training-b/', 'training-c/', 'training-d/', 'training-e/', 'training-f/'
allFiles = True

# for i in range(len(folderList)):
for i in range(5,6):
    fname = folderList[i].name
    
    inFolder = inputPath / fname
    outFolder = outputPath / fname
    outFolder.mkdir(parents=True, exist_ok=True)
    
    segInFolder1 = segPath1 / fname
    fname = fname + '-Aut'
    segInFolder2 = segPath2 / fname
    
    fileList = sorted(list(inFolder.glob('*.wav')))
    segFileList1 = sorted(list(segInFolder1.glob('*.mat')))
    segFileList2 = sorted(list(segInFolder2.glob('*.mat')))
    
#     fileList = fileList[1229:]

    j = 0
    for k in range(len(segFileList1)):  # Porque training-e tem menos segmentações do que arquivos .wav, 
        # verificar se correspondem
#         print(segFileList1[j].stem.rsplit('_')[0], fileList[k].stem)
        if(segFileList1[j].stem.rsplit('_')[0] != fileList[k].stem):
            pass
        else:
        
            recordName = inFolder / fileList[k]
            outFile = (outFolder / fileList[k].stem).with_suffix('.npz')

            # Verifica se tem hand corrected, caso contrário usa automated
            if (allFiles):
                if (quality[j]=='Ok'):
                    segName = segInFolder1 / segFileList1[k]
                    key = 'state_ans'

                else:
                    segName = segInFolder2 / segFileList2[k]
                    key = 'state_ans0'

            else:
                segName = segInFolder1 / segFileList1[k]
            
            j+=1

            print(f'{folderList[i].name} | File {k+1} of {len(fileList)} | {fileList[k].stem}')
            

            # Load data and resample data
            PCG_resampled = librosa.load(recordName,sr=springer_options.audio_Fs)[0]

            # Segmentador
            assigned_states = runSpringerSegmentationAlgorithm(PCG_resampled, model)
            # Transforma vetor de estados para o formato seg_data (dicionário do Python com índices e nome dos estados)
            seg_data = gsd.getdata(assigned_states)
            # Salva
            np.savez(outFile, **seg_data)
            # Pega seg_data do segPath
            seg_data = scio.loadmat(segName, struct_as_record=False, squeeze_me=True)
            # converte do formato seg_data para vetor de estados
            
            states = get_states(seg_data, assigned_states.size, key)
            # salva figuras com os plots do arquivo de áudio e segmentações
            spath = figsPath / folderList[i].name / ('assigned_states_' + fileList[k].stem)
            save_figs(PCG_resampled, springer_options.audio_Fs, assigned_states, states, spath, 0, 10)

training-f | File 1 of 114 | f0001
training-f | File 2 of 114 | f0002
training-f | File 3 of 114 | f0003
training-f | File 4 of 114 | f0004
training-f | File 5 of 114 | f0005
training-f | File 6 of 114 | f0006
training-f | File 7 of 114 | f0007
training-f | File 8 of 114 | f0008
training-f | File 9 of 114 | f0009
training-f | File 10 of 114 | f0010
training-f | File 11 of 114 | f0011
training-f | File 12 of 114 | f0012
training-f | File 13 of 114 | f0013
training-f | File 14 of 114 | f0014
training-f | File 15 of 114 | f0015
training-f | File 16 of 114 | f0016
training-f | File 17 of 114 | f0017
training-f | File 18 of 114 | f0018
training-f | File 19 of 114 | f0019
training-f | File 20 of 114 | f0020
training-f | File 21 of 114 | f0021
training-f | File 22 of 114 | f0022
training-f | File 23 of 114 | f0023
training-f | File 24 of 114 | f0024
training-f | File 25 of 114 | f0025
training-f | File 26 of 114 | f0026
training-f | File 27 of 114 | f0027
training-f | File 28 of 114 | f0028
t